In [1]:
import pandas as pd
df = pd.read_csv('Horse_Win.csv')
df.head()

,Tipster,Date,Track,Horse,Bet Type,Odds,Result,TipsterActive
0,Tipster A,24-07-2015,Ascot,Fredricka,Win,8.00,Lose,True
1,Tipster A,24-07-2015,Thirsk,Spend A Penny,Win,4.50,Lose,True
2,Tipster A,24-07-2015,York,Straightothepoint,Win,7.00,Lose,True
3,Tipster A,24-07-2015,Newmarket,Miss Inga Sock,Win,5.00,Lose,True
4,Tipster A,25-07-2015,Ascot,Peril,Win,4.33,Win,True


In [2]:
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

In [4]:
import numpy as np
df['date_year'] = df['Date'].dt.year
df['date_month_no'] = df['Date'].dt.month
df['date_day'] = df['Date'].dt.day
df['date_dow'] = df['Date'].dt.dayofweek
df['date_is_weekend'] = np.where(df['date_dow'].isin([0,1]), 1,0)
df

,Tipster,Date,Track,Horse,Bet Type,Odds,Result,TipsterActive,date_year,date_month_no,date_day,date_dow,date_is_weekend
0,Tipster A,2015-07-24,Ascot,Fredricka,Win,8.00,Lose,True,2015,7,24,4,0
1,Tipster A,2015-07-24,Thirsk,Spend A Penny,Win,4.50,Lose,True,2015,7,24,4,0
2,Tipster A,2015-07-24,York,Straightothepoint,Win,7.00,Lose,True,2015,7,24,4,0
3,Tipster A,2015-07-24,Newmarket,Miss Inga Sock,Win,5.00,Lose,True,2015,7,24,4,0
4,Tipster A,2015-07-25,Ascot,Peril,Win,4.33,Win,True,2015,7,25,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38243,Tipster E1,2016-04-02,Kempton,Solar Flair,Win,7.00,Lose,False,2016,4,2,5,0
38244,Tipster E1,2016-04-02,Doncaster,Express Himself,Each Way,12.00,Lose,False,2016,4,2,5,0
38245,Tipster E1,2016-04-02,Doncaster,Jack Dexter,Win,7.00,Lose,False,2016,4,2,5,0
38246,Tipster E1,2016-04-02,Kelso,Just Cameron,Win,4.33,Lose,False,2016,4,2,5,0


In [5]:
df = df.drop(columns=['Date'],axis=1)

In [6]:
df

,Tipster,Track,Horse,Bet Type,Odds,Result,TipsterActive,date_year,date_month_no,date_day,date_dow,date_is_weekend
0,Tipster A,Ascot,Fredricka,Win,8.00,Lose,True,2015,7,24,4,0
1,Tipster A,Thirsk,Spend A Penny,Win,4.50,Lose,True,2015,7,24,4,0
2,Tipster A,York,Straightothepoint,Win,7.00,Lose,True,2015,7,24,4,0
3,Tipster A,Newmarket,Miss Inga Sock,Win,5.00,Lose,True,2015,7,24,4,0
4,Tipster A,Ascot,Peril,Win,4.33,Win,True,2015,7,25,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...
38243,Tipster E1,Kempton,Solar Flair,Win,7.00,Lose,False,2016,4,2,5,0
38244,Tipster E1,Doncaster,Express Himself,Each Way,12.00,Lose,False,2016,4,2,5,0
38245,Tipster E1,Doncaster,Jack Dexter,Win,7.00,Lose,False,2016,4,2,5,0
38246,Tipster E1,Kelso,Just Cameron,Win,4.33,Lose,False,2016,4,2,5,0


In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Result']),df['Result'],test_size=0.2,random_state=42)

In [9]:
df['Bet Type'] = df['Bet Type'].replace("win","Win")

In [10]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
y_trans = label_encoder.fit_transform(y_train)

In [11]:
def encode_column(column):
    series_column = column.squeeze()  
    enc_column = series_column.map(series_column.value_counts() / len(series_column))
    return enc_column.to_frame()

In [14]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
trf1 = ColumnTransformer([
    ('Horse_encode', FunctionTransformer(encode_column, validate=False), ['Horse']),
    ('Track_encode', FunctionTransformer(encode_column, validate=False), ['Track']),
    ('Tipster_encode', FunctionTransformer(encode_column, validate=False), ['Tipster']),
    ('ohe',OneHotEncoder(handle_unknown='ignore'),[3,6]),    
], remainder ='passthrough')

In [42]:
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
dec_tree = tree.DecisionTreeClassifier(criterion='entropy', max_depth=5)

In [43]:
pipe = Pipeline([
    ('trf1',trf1),
    ('dec',dec_tree)
])

In [44]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Horse_encode',
                                                  FunctionTransformer(func=<function encode_column at 0x000001958DFD5EE0>),
                                                  ['Horse']),
                                                 ('Track_encode',
                                                  FunctionTransformer(func=<function encode_column at 0x000001958DFD5EE0>),
                                                  ['Track']),
                                                 ('Tipster_encode',
                                                  FunctionTransformer(func=<function encode_column at 0x000001958DFD5EE0>),
                                                  ['Tipster']),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  [3, 6])])),
                ('dec',
                 DecisionTreeClassifier(criterion='entropy', max_depth=5))])

In [45]:
from sklearn import set_config
set_config(display='diagram')

In [46]:
y_pred = pipe.predict(X_test)

In [47]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8009150326797385

In [48]:
y_pred1 = pipe.predict(X_train)

In [49]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train,y_pred1)

0.8005098372442643

In [50]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

0.7983202325981327

In [53]:
param_grid = {
    "dec__criterion": ['gini', 'entropy'],
    "dec__max_depth": [2,4,6,8,10,12]
}

In [54]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('Horse_encode',
                                                                         FunctionTransformer(func=<function encode_column at 0x000001958DFD5EE0>),
                                                                         ['Horse']),
                                                                        ('Track_encode',
                                                                         FunctionTransformer(func=<function encode_column at 0x000001958DFD5EE0>),
                                                                         ['Track']),
                                                                        ('Tipster_encode',
                                                                         FunctionTransformer(func=<function encode_column at 0x000001958DFD5EE0>),
                                                                         ['Tipster']),
                                                                        ('ohe',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         [3,
                                                                          6])])),
                                       ('dec',
                                        DecisionTreeClassifier(criterion='entropy',
                                                               max_depth=5))]),
             param_grid={'dec__criterion': ['gini', 'entropy'],
                         'dec__max_depth': [2, 4, 6, 8, 10, 12]},
             scoring='accuracy')

In [55]:
from sklearn import set_config
set_config(display='diagram')

In [56]:
grid.best_score_

0.7991699592077393

In [57]:
grid.best_params_

{'dec__criterion': 'gini', 'dec__max_depth': 4}

In [58]:
import pickle
pickle.dump(pipe,open('dec_pipe.pkl','wb'))